In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import settings
import requests
import json

In [29]:
API_KEY = settings.PLANTNET_API_KEY
PROJECT = 'all'
api_endpoint = f"https://my-api.plantnet.org/v2/identify/{PROJECT}?api-key={API_KEY}"

# Identifying plants

We'll use the PlantNet API (as a starting point, TODO: investigate iNaturalist API) to identify user-uploaded images of plants. As a first step, we'll expect the user photos to contain a single plant, though eventually we want to apply image segmentation to handle whole-landscape images. The PlantNet API requires us to identify a prominent organ type (leaf, flower, bark, etc.) in the image to aid in identification; we need to experiment with how much this matters, and whether we can handle it without user input.

In [30]:
img_path = '../data/img/img1.jpg'
with open(img_path, 'rb') as img_data:
    files = [
            ('images', (img_path,img_data))
            ]
    req = requests.Request('POST', url = api_endpoint, files = files, data = {'organs': ['flower',]})
    prepared = req.prepare()
    s = requests.Session()
    response = s.send(prepared)
    result = json.loads(response.text)

In [31]:
print(result['bestMatch'], result['results'][0]['score'], result['results'][0]['species']['commonNames'])

Pyrus calleryana Decne. 0.42614 ['Bradford Pear', 'Callery pear', 'Ornamental pear']


Let's wrap this in a function.

In [86]:
def id_plant(img_path,organ = 'leaf'):
    with open(img_path, 'rb') as img_data:
        files = [
                ('images', (img_path,img_data))
                ]
        req = requests.Request('POST', url = api_endpoint, files = files, data = {'organs': [organ,]})
        prepared = req.prepare()
        s = requests.Session()
        response = s.send(prepared)
        result = json.loads(response.text)
    return result['bestMatch'], result['results'][0]['species']['commonNames'], result['results'][0]['score']

def species_id(plant_id):
    species = plant_id[0].lower().split()
    return ''.join(species[0] + ' ' + species[1])

id_plant(img_path), species_id(id_plant(img_path))

(('Pyrus calleryana Decne.',
  ['Bradford Pear', 'Callery pear', 'Ornamental pear'],
  0.42614),
 'pyrus calleryana')

# Comparing results to the PA Invasive species list

In [16]:
invasives = pd.read_csv('../data/pa_invasives.csv')
invasives.head(5)

,Scientific Name,Common Name,PISC Priority Score,PA Noxious Weed Rank,Invasive Assessment Score,Aquatic or Terrestrial,PLNA Economic Importance Score,DCNR Rank,EDRR
0,Wisteria sinensis,Chinese Wisteria,3.6,NaN,57.0,Terrestrial,4.2,2,No
1,Wisteria floribunda,Japanese Wisteria,4.5,NaN,57.0,Terrestrial,4.6,2,No
2,Vincetoxicum rossicum,Pale Swallow-Wort,4.4,B,88.0,Terrestrial,0.1,1,No
3,Vincetoxicum nigrum,Black Swallow-Wort,4.5,B,90.0,Terrestrial,0.4,1,No
4,Vinca minor,Common Periwinkle,2.2,NaN,57.0,Terrestrial,7.6,3,No


In [17]:
invasives.iloc[92,0]

'reynoutria japonica'

In [10]:
invasives.iloc[92,0] = 'reynoutria japonica'
invasives.to_csv('../data/pa_invasives.csv',index=False)

C:\Users\mreek\AppData\Local\Temp\ipykernel_16896\2497986613.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'reynoutria japonica' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  invasives.iloc[92,0] = 'reynoutria japonica'


We've identified an invasive Callery/Bradford pear up above. Let's try to match it to this list.

In [34]:
invasives.loc[invasives['Scientific Name'] == 'pyrus calleryana']

,Scientific Name,Common Name,PISC Priority Score,PA Noxious Weed Rank,Invasive Assessment Score,Aquatic or Terrestrial,PLNA Economic Importance Score,DCNR Rank,EDRR


In [15]:
'reynoutria japonica' in invasives['Scientific Name'].to_list()

True

In [13]:
invasives['Scientific Name'] = invasives['Scientific Name'].str.lower()


invasives[invasives['Scientific Name'] == species_id(id_plant(img_path))]

NameError: name 'species_id' is not defined

In [80]:
def is_invasive(species_id):
    if species_id in invasives['Scientific Name'].str.lower().to_list():
        return 'Invasive'
    else:
        return 'Non-invasive'

Great! Let's do some battery testing.

In [95]:
import os

imgs = os.listdir('../data/img')
for img in imgs:
    plant_id = id_plant('../data/img/'+img) #organ always set to leaf -- to what extent does this matter?
    species = species_id(plant_id)
    invasive = is_invasive(species)
    print(f'ID: {species}, common names {plant_id[1]} is {invasive}.')


ID: pyrus calleryana, common names ['Bradford Pear', 'Callery pear', 'Ornamental pear'] is Invasive.
ID: wisteria sinensis, common names ['Chinese Wisteria', 'Purple wisteria', 'پیچ گلیسین'] is Invasive.
ID: reynoutria japonica, common names ['Japanese knotweed', 'Impossible to kill invasive demon plant', 'Mexican-bamboo'] is Invasive.
ID: viburnum acerifolium, common names ['Mapleleaf viburnum', 'Dockmackie', 'Arrow-wood'] is Non-invasive.
ID: tulipa agenensis, common names ['Eyed tulip', 'Common tulip', 'Tulip'] is Non-invasive.
ID: quercus robur, common names ['Common Oak', 'English oak', 'Carvalho-alvarinho'] is Non-invasive.
ID: phlox subulata, common names ['Moss phlox', 'Creeping phlox', 'Moss-pink'] is Non-invasive.
